In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv(r'C:\Users\edgar.manzo\Desktop\training dadp\capstone\data input\layoffs_1.csv')
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,ResearchGate,Berlin,Other,25,0.10,11/28/2022,Series D,Germany,87.0
1,VerSe Innovation,Bengaluru,Media,150,0.05,11/25/2022,Series J,India,1700.0
2,Lemon,Buenos Aires,Crypto,100,0.38,11/24/2022,Series A,Argentina,17.0
3,Quidax,Lagos,Crypto,20,0.20,11/24/2022,Unknown,Nigeria,3.0
4,Menulog,Sydney,Food,0,0.00,11/24/2022,Acquired,Australia,0.0


In [3]:
def plot_layoffs_over_time(parameter, aggfunc, time_period='M'):
    df.date = pd.to_datetime(df.date)
    df['period'] = df.date.dt.to_period(time_period)
    df.period = df.period.dt.to_timestamp()

    # get agg values of parameter by period
    grouped = df.groupby(['period']).agg(agg_val = (parameter, aggfunc)).reset_index()
    
    # plot
    time_map = {'M':'Month', 'Q':'Quarter', 'W':'Week', 'Y':'Year'}
    func_map = {'sum': 'Total', 'mean':'Average'}
    fig = px.area(grouped, x='period', y='agg_val',
                  labels = {'value' : parameter},
                  title='%s %s by %s' % (func_map[aggfunc], parameter, time_map[time_period]))
    return fig

In [4]:
df['date']= pd.to_datetime(df['date'])
df['bulan']= df['date'].dt.to_period('M')
df['bulan']= df['bulan'].dt.to_timestamp()

In [5]:
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised,bulan
0,ResearchGate,Berlin,Other,25,0.10,2022-11-28,Series D,Germany,87.0,2022-11-01
1,VerSe Innovation,Bengaluru,Media,150,0.05,2022-11-25,Series J,India,1700.0,2022-11-01
2,Lemon,Buenos Aires,Crypto,100,0.38,2022-11-24,Series A,Argentina,17.0,2022-11-01
3,Quidax,Lagos,Crypto,20,0.20,2022-11-24,Unknown,Nigeria,3.0,2022-11-01
4,Menulog,Sydney,Food,0,0.00,2022-11-24,Acquired,Australia,0.0,2022-11-01


In [8]:
#Area Plot
group = pd.pivot_table(df, index='bulan', values='total_laid_off', aggfunc='sum').reset_index()

area_plot = px.area(group, 
        x='bulan', 
        y='total_laid_off', 
        labels= {'bulan': 'Month', 'total_laid_off':'Number of people laid off'},
       template= 'ggplot2',
       title='Number of People Laid Off in Months')


In [64]:
df['status'] = ['Publicly traded' if x =='IPO' else 'Privately owned' for x in df['stage']]
# Pie plot
status = pd.pivot_table(df, 
                            index='status', 
                            values='total_laid_off', 
                            aggfunc='sum').sort_values('total_laid_off',ascending=False).reset_index()
fig = px.pie(status.sort_values('total_laid_off', ascending=True), 
       values='total_laid_off', 
       names='status',
      title=f"Most layoffs came from {str(status['status'].head(1)[0])} companies",
      template='ggplot2')
fig.update_traces(hovertemplate='<b>%{value}</b> employees have been laid off from <b>%{label}</b> companies',
textinfo='label+percent')
fig.show()

In [77]:
# Bar plot 1

industry = pd.pivot_table(df, 
                            index='industry', 
                            values='total_laid_off', 
                            aggfunc='sum').sort_values('total_laid_off',ascending=False).reset_index().head(10)
bar_1 = px.bar(industry.sort_values('total_laid_off', ascending=True), 
       x='total_laid_off', 
       y='industry',
      title='Top 10 Industry with most layoffs',
      template='ggplot2',
      labels= {'industry': 'Industry', 'total_laid_off':'Number of people laid off'})


In [70]:
#Bar plot 2
perusahaan = pd.pivot_table(df, 
                            index='company', 
                            values='total_laid_off', 
                            aggfunc='sum').sort_values('total_laid_off',ascending=False).reset_index().head(10)
bar_2 = px.bar(perusahaan.sort_values('total_laid_off', ascending=True), 
       x='total_laid_off', 
       y='company',
      title='Top 10 Company with most layoffs',
      template='ggplot2',
      labels= {'company': 'Company', 'total_laid_off':'Number of people laid off'})

In [63]:
sketer = px.scatter(df[(df['total_laid_off'] != 0) & (df['funds_raised'] != 0)], 'funds_raised', 'total_laid_off',
          template='ggplot2', log_x=True, hover_data=['company'],
          title = 'There is a positive, albeit weak correlation between log of funds raised by companies and number of people laid off',
          labels= {'funds_raised': 'Log(funds raised)', 'total_laid_off':'Number of people laid off'})
sketer.update_traces(hovertemplate='% company have laid off %{y} employees have raised <b>%{x}</b> Mio USD')

In [75]:
# bar plot 3

lokasi = pd.pivot_table(df, 
                            index='location', 
                            values='total_laid_off', 
                            aggfunc='sum').sort_values('total_laid_off',ascending=False).reset_index().head(10)
bar_3 = px.bar(lokasi.sort_values('total_laid_off', ascending=True), 
       x='total_laid_off', 
       y='location',
      title='Top 10 cities with most layoffs',
      template='ggplot2',
      labels= {'location': 'City', 'total_laid_off':'Number of people laid off'})

In [44]:
import numpy as np
tes = df[['total_laid_off', 'funds_raised']]
tes['log_funds'] = np.log(df['funds_raised'])

tes.corr()

c:\Users\edgar.manzo\Anaconda3\envs\smm_dadp\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning:

divide by zero encountered in log

C:\Users\edgar.manzo\AppData\Local\Temp\ipykernel_23344\791799205.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,total_laid_off,funds_raised,log_funds
total_laid_off,1.000000,0.121606,0.230865
funds_raised,0.121606,1.000000,0.341562
log_funds,0.230865,0.341562,1.000000


In [ ]:
# your code here
# Praqualifikasi Vendor

agg_vendor_PraQ = pd.crosstab(
    index=iss['prakualifikasi'],
    columns='jumlah vendor'
).reset_index().sort_values(by = 'jumlah vendor', ascending = True)

agg_vendor_PraQ["all"] = "Prakualifikasi Vendor" # in order to have a single root node
fig = px.treemap(
    agg_vendor_PraQ, 
    path=['all', 'prakualifikasi'], 
    values='jumlah vendor',
    color_discrete_sequence =['#ff6600','salmon','#ff3333' ,'#ff9966']*len(agg_vendor_PraQ),
    )

fig.update_traces(
    root_color="lightgrey",
    hovertemplate='<b> %{label} </b> <br> Total Vendor: %{value}'
    )
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()
